In [1]:
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision
from torchvision import models
import torch.optim as optim
import pandas as pd
import numpy as np
import cv2
import os
from sklearn import preprocessing
import matplotlib.pyplot as plt
import re
import pickle

In [ ]:
!unzip /content/drive/MyDrive/CelebrityFaceRecognition/zipped/CombinedReduced.zip -d /content/drive/MyDrive/CelebrityFaceRecognition/zipped/

In [2]:
def initialize_celeb_face_recog():
    global output_file_path, output_low_level_path, list_ref_persons, ref_file_path
    output_file_path = "/content/drive/MyDrive/project-sepnu/data/simile classifier/celeb_face_recog/"
    output_low_level_path = "/content/drive/MyDrive/project-sepnu/data/low level/celeb_face_recog/"
    ref_file_path = "/content/drive/MyDrive/CelebrityFaceRecognition/reference_faces/"
    file_ref_persons = open(ref_file_path + "reference_faces.txt", "r")
    list_ref_persons = [ref_person.replace("\n", "") for ref_person in file_ref_persons.readlines()]
    file_ref_persons.close()

def learn_SVMs():
    global output_file_path, output_low_level_path, list_ref_persons, ref_file_path
    for ref_person in list_ref_persons:
        for attribute, features in tqdm(dict_facepart_features.items()):
            final_feature_set = np.empty(shape = (np.load(output_low_level_path + features[0] + ".npy").shape[0], 0))
            for feature in features:
                feature_array = np.load(output_low_level_path + feature + ".npy")
                final_feature_set = np.append(final_feature_set, feature_array / feature_array.sum(axis = 1).reshape(-1, 1), axis = 1)
            labels = np.load(ref_file_path + ref_person + "_labels.npy")

            indices = (labels == -1)
            X_train_1, X_test_1, y_train_1, y_test_1 = model_selection.train_test_split(final_feature_set[indices], labels[indices], test_size = 0.3)
            indices = (labels == 1)
            X_train_2, X_test_2, y_train_2, y_test_2 = model_selection.train_test_split(final_feature_set[indices], labels[indices], test_size = 0.15)
            X_train = np.append(X_train_1, X_train_2, axis = 0)
            X_test = np.append(X_test_1, X_test_2, axis = 0)
            y_train = np.append(y_train_1, y_train_2)
            y_test = np.append(y_test_1, y_test_2)
            np.save(output_file_path + ref_person + "_" + attribute + "_X_train", X_train)
            np.save(output_file_path + ref_person + "_" + attribute + "_y_train", y_train)
            np.save(output_file_path + ref_person + "_" + attribute + "_X_test", X_test)
            np.save(output_file_path + ref_person + "_" + attribute + "_y_test", y_test)

            svm_obj = svm.SVC()
            grid_values = {'C':[0.01, 0.1, 1, 5, 10, 15, 20, 25, 30, 50]}
            grid_clf_acc = model_selection.GridSearchCV(svm_obj, param_grid = grid_values, scoring = 'accuracy')
            grid_clf_acc.fit(X_train, y_train)
            with open(output_file_path + ref_person + "_" + attribute + "_svm_model.pkl", "wb") as f: pickle.dump(grid_clf_acc, f)

def test_SVMs():
    global output_file_path
    for ref_person in list_ref_persons:
        for attribute in dict_facepart_features:
            with open(output_file_path + ref_person + "_" + attribute + "_svm_model.pkl", "rb") as f: grid_clf_acc = pickle.load(f)
            predicted_output = grid_clf_acc.predict(np.load(output_file_path + ref_person + "_" + attribute + "_X_test.npy"))
            print("----------For reference person %s and attribute %s----------" % (ref_person, attribute))
            print(metrics.classification_report(np.load(output_file_path + ref_person + "_" + attribute + "_y_test.npy"), predicted_output))

In [3]:
class CelebRecogData(Dataset):
    def __init__(self,data_list,map_name_label,data_dir,transform=None,train=True):
        super().__init__()
        self.data_list = data_list
        self.data_dir = data_dir
        self.transform = transform
        self.train = train
        self.map_name_label = map_name_label

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self,item):
        img_idx = item
        imgname = self.data_list[item]
        celeb_name = imgname[0]
        img_num = imgname[1]
        imgpath = os.path.join(self.data_dir,celeb_name,img_num)
        img = cv2.imread(imgpath)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        imgr = cv2.resize(img,(224,224))
        label = self.map_name_label[celeb_name]
        if self.transform is not None:
            imgr = self.transform(imgr)
        return {'img' : imgr,'label' : torch.tensor(label)}

In [15]:
initialize_celeb_face_recog()
print(list_ref_persons)
path = "/content/drive/MyDrive/CelebrityFaceRecognition/zipped"
map_name_label = {list_ref_persons[i]:i for i in range(len(list_ref_persons))}
datalist = []
labellist = []
for name in list_ref_persons:
    name_path = os.path.join(path,name)
    name_img_list = os.listdir(name_path)
    for file_name in name_img_list:
        if file_name.endswith(".jpg"):
            datalist.append([name,file_name])
            labellist.append(map_name_label[name])
        # else :
            # print("exception occured!",name,file_name)


# print(len(datalist))
# print(labellist)
for person in os.listdir(path):
    print(person)

['Aaron_Taylor-Johnson', 'Adriana_Barraza', 'Alec_Baldwin', 'Ali_Larter', 'Amber_Heard', 'Andie_MacDowell', 'Andrew_Lincoln', 'Angelina_Jolie', 'Anne_Hathaway', 'Benedict_Cumberbatch', 'Betty_White', 'Beyonce_Knowles']
Extract.ipynb
Amber_Heard
Beyonce_Knowles
Bill_O_Reilly
Amy_Adams
Ali_Larter
Anne_Hathaway
Bella_Thorne
Ben_Affleck
Arnold_Schwarzenegger
Benedict_Cumberbatch
Andrew_Lincoln
Anna_Friel
Al_Pacino
Ben_Stiller
Angelina_Jolie
Bella_Hadid
Anderson_Cooper
Anna_Faris
Andrew_Garfield
Betty_White
Amanda_Crew
Andie_MacDowell
Anna_Kendrick
CombinedReduced.zip
Aaron_Judge
Aaron_Paul
Aaron_Taylor-Johnson
Abigail_Breslin
Adam_Sandler
Adele
Adriana_Barraza
Adriana_Lima
Adrianne_Palicki
Adrien_Brody
Alan_Arkin
Alec_Baldwin
Alessandra_Ambrosio
Alexandra_Daddario


In [5]:
transforms_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.3166, 0.3947, 0.4725), (0.1755, 0.1720, 0.1657)),
])
dataset = CelebRecogData(datalist,map_name_label,path,transforms)
# imglabel = dataset.__getitem__(3)
# plt.imshow(imglabel["img"])
# print(imglabel["label"])

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:


print(device)

cuda:0


In [7]:
def get_resnet():
    model1 = models.resnet50(pretrained=True)
    for name, child in model1.named_children():
        for param in child.parameters():
            param.requires_grad = False
    # Parameters of newly constructed modules have requires_grad=True by default
    num_ftrs = model1.fc.in_features
    model1.fc = nn.Sequential(
                nn.Linear(num_ftrs, 1024),
                nn.ReLU(inplace=True),
                nn.Linear(1024, 512),
                nn.ReLU(inplace=True),
                nn.Linear(512, len(list_ref_persons)))
    return model1
# model = model1.to(device)

In [8]:
model = get_resnet().to(device)
rand_tensor = 8*torch.rand((64,3,224,224))
temp = rand_tensor.to(device)
#print tensor
# print(rand_tensor)
y = model(temp)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [9]:
from tqdm.notebook import tqdm
def train_test(data_list,data_dir,transforms=None):
    # full_dataset = CustomData(transforms,train_path,trainimgs)
    # print("=========================================================================================")
    # print("Current Attribute:", attr)
    full_dataset = CelebRecogData(datalist,map_name_label,path,transforms)

    train_size = int(0.8 * len(full_dataset)) 
    test_size = len(full_dataset) - train_size

    batch = 32
    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=False)
    model = get_resnet()
    model = model.to(device)
    optimizer = optim.Adam(model.parameters())
    error = nn.CrossEntropyLoss().to(device)
    num_epochs = 15
    valid_loss_min = np.Inf

    losses = {'train' : [] } 
    accuracies = {'train' : []} 
    dataloaders = {
        'train':train_loader,
        'test':test_loader
    }

    model.train()
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        curr_loss = 0.0
        curr_acc = 0

        for dinputs in tqdm(dataloaders["train"]):
            inputs = dinputs["img"].to(device)
            labels = dinputs["label"].to(device)

            outputs = model(inputs)
            loss = error(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            curr_loss += loss.item() * inputs.size(0)
            curr_acc += torch.sum(preds == labels.data)

        curr_loss = curr_loss / len(dataloaders["train"].sampler)
        curr_acc = curr_acc.double() / len(dataloaders["train"].sampler)
        
        losses["train"].append(curr_loss)
        accuracies["train"].append(curr_acc)
        # if epoch == num_epochs - 1:
        print("train" + ":")
        print('loss = {:.4f}     accuracy = {:.4f}'.format(curr_loss,curr_acc))
            # print()
        # train_losses.append(train_loss)
        # valid_losses.append(valid_loss)

    '''
    test code here
    '''
    model.eval()
    test_loss = 0.0
    test_acc = 0
    for dinputs in dataloaders["test"]:
        inputs = dinputs["img"].to(device)
        labels = dinputs["label"].to(device)

        outputs = model(inputs)
        loss = error(outputs, labels)

        _, preds = torch.max(outputs, 1)
        test_loss += loss.item() * inputs.size(0)
        test_acc += torch.sum(preds == labels.data)

    test_loss = test_loss / len(dataloaders["test"].sampler)
    test_acc = test_acc.double() / len(dataloaders["test"].sampler)
    print("test" + ":")
    print("loss: {:.4f}     accuracy: {:.4f}".format(test_loss, test_acc))

    return losses,accuracies    

In [10]:
loss,acc = train_test(datalist,path,transforms_train)

Epoch 1/15
----------



train:
loss = 1.8827     accuracy = 0.2938
Epoch 2/15
----------



train:
loss = 1.2077     accuracy = 0.5470
Epoch 3/15
----------



train:
loss = 0.9410     accuracy = 0.6592
Epoch 4/15
----------



train:
loss = 0.7597     accuracy = 0.7253
Epoch 5/15
----------


...


train:
loss = 0.3136     accuracy = 0.8844
Epoch 12/15
----------



train:
loss = 0.2626     accuracy = 0.9109
Epoch 13/15
----------



train:
loss = 0.3019     accuracy = 0.8937
Epoch 14/15
----------



train:
loss = 0.2783     accuracy = 0.9011
Epoch 15/15
----------



train:
loss = 0.2107     accuracy = 0.9216
test:
loss: 0.6043     accuracy: 0.7965
